# Data Prep

In [1]:
# Common Imports
import numpy as np
import pandas as pd
np.random.seed(1)


In [2]:
#Reading the data set:

hc = pd.read_csv("healthcare.csv")
hc.head()

,race,gender,age,admission_type,discharge_disposition,admission_source,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,...,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,insulin,change,diabetesMed,readmitted
0,Other,Female,70-80,2,3,1,14,NaN,InternalMedicine,32,...,486,404,428,9,None,None,No,No,No,1
1,Caucasian,Female,80-90,1,3,5,4,MC,NaN,44,...,38,438,599,9,None,None,Steady,Ch,Yes,0
2,AfricanAmerican,Male,50-60,5,1,1,6,HM,NaN,29,...,296,585,428,9,None,None,Up,Ch,Yes,1
3,Caucasian,Female,50-60,1,1,6,3,HM,InternalMedicine,47,...,250.02,401,493,4,None,>8,No,Ch,Yes,0
4,AfricanAmerican,Female,40-50,3,1,1,4,UN,Hematology,92,...,486,287,595,7,None,>7,No,No,No,0


In [3]:
# Importing the train_test_split and creating the split

from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(hc, test_size=0.3)

In [4]:
train_set.isna().sum()

race                      117
gender                      0
age                         0
admission_type              0
discharge_disposition       0
admission_source            0
time_in_hospital            0
payer_code               2486
medical_specialty        3037
num_lab_procedures          0
num_procedures              0
num_medications             0
number_outpatient           0
number_emergency            0
number_inpatient            0
diag_1                      2
diag_2                     14
diag_3                     64
number_diagnoses            0
max_glu_serum               0
A1Cresult                   0
insulin                     0
change                      0
diabetesMed                 0
readmitted                  0
dtype: int64

In [5]:
test_set.isna().sum()

race                       57
gender                      0
age                         0
admission_type              0
discharge_disposition       0
admission_source            0
time_in_hospital            0
payer_code               1038
medical_specialty        1312
num_lab_procedures          0
num_procedures              0
num_medications             0
number_outpatient           0
number_emergency            0
number_inpatient            0
diag_1                      1
diag_2                      6
diag_3                     34
number_diagnoses            0
max_glu_serum               0
A1Cresult                   0
insulin                     0
change                      0
diabetesMed                 0
readmitted                  0
dtype: int64

In [6]:
#Imports for Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.preprocessing import FunctionTransformer

In [7]:
#Seperaring the Y Variable
train_y = train_set['readmitted']
test_y = test_set['readmitted']

train_inputs = train_set.drop(['readmitted'], axis=1)
test_inputs = test_set.drop(['readmitted'], axis=1)

## Feature Engineering

In [8]:

train_inputs['per_day_diagnosis']= train_inputs['number_diagnoses']/ train_inputs['time_in_hospital']
test_inputs['per_day_diagnosis']= test_inputs['number_diagnoses']/ test_inputs['time_in_hospital']
train_inputs['total_procedures']= train_inputs['num_lab_procedures']+train_inputs['num_procedures']
test_inputs['total_procedures']= test_inputs['num_lab_procedures']+test_inputs['num_procedures']
train_inputs['total_visits']= train_inputs['number_outpatient']+train_inputs['number_emergency']+train_inputs['number_inpatient']
test_inputs['total_visits']= test_inputs['number_outpatient']+test_inputs['number_emergency']+test_inputs['number_inpatient']

In [9]:
##  Identifying the numerical and categorical columns

In [10]:
train_inputs.dtypes

race                      object
gender                    object
age                       object
admission_type             int64
discharge_disposition      int64
admission_source           int64
time_in_hospital           int64
payer_code                object
medical_specialty         object
num_lab_procedures         int64
num_procedures             int64
num_medications            int64
number_outpatient          int64
number_emergency           int64
number_inpatient           int64
diag_1                    object
diag_2                    object
diag_3                    object
number_diagnoses           int64
max_glu_serum             object
A1Cresult                 object
insulin                   object
change                    object
diabetesMed               object
per_day_diagnosis        float64
total_procedures           int64
total_visits               int64
dtype: object

In [11]:
# Identifying the numerical columns
numeric_columns = train_inputs.select_dtypes(include=[np.number]).columns.to_list()

# Identifying the categorical columns
categorical_columns = train_inputs.select_dtypes('object').columns.to_list()

In [12]:
# Identifying the binary columns so we can pass them through without transforming
binary_columns = []

In [13]:
for col in binary_columns:
    numeric_columns.remove(col)

In [14]:
binary_columns

[]

In [15]:
numeric_columns

['admission_type',
 'discharge_disposition',
 'admission_source',
 'time_in_hospital',
 'num_lab_procedures',
 'num_procedures',
 'num_medications',
 'number_outpatient',
 'number_emergency',
 'number_inpatient',
 'number_diagnoses',
 'per_day_diagnosis',
 'total_procedures',
 'total_visits']

In [16]:
categorical_columns

['race',
 'gender',
 'age',
 'payer_code',
 'medical_specialty',
 'diag_1',
 'diag_2',
 'diag_3',
 'max_glu_serum',
 'A1Cresult',
 'insulin',
 'change',
 'diabetesMed']

## Pipeline

In [17]:
numeric_transformer = Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())])

In [18]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [19]:
binary_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent'))])

In [20]:
preprocessor = ColumnTransformer([
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns),
        ('binary', binary_transformer, binary_columns)],
        remainder='passthrough')



In [21]:
#Fit and transformation of train data
train_x = preprocessor.fit_transform(train_inputs)

train_x

<6066x1287 sparse matrix of type '<class 'numpy.float64'>'
	with 163782 stored elements in Compressed Sparse Row format>

In [22]:
train_x.shape

(6066, 1287)

In [23]:
# Transform of test data
test_x = preprocessor.transform(test_inputs)

test_x

<2600x1287 sparse matrix of type '<class 'numpy.float64'>'
	with 70041 stored elements in Compressed Sparse Row format>

In [24]:
test_x.shape

(2600, 1287)

# Baseline:

In [25]:
train_y.value_counts()/len(train_y)

0    0.536268
1    0.463732
Name: readmitted, dtype: float64

# Decision Tree Model 1:

In [26]:
from sklearn.tree import DecisionTreeClassifier 

tree_clf = DecisionTreeClassifier(min_samples_leaf=12)

tree_clf.fit(train_x, train_y)

DecisionTreeClassifier(min_samples_leaf=12)

In [27]:
from sklearn.metrics import accuracy_score

In [28]:
#Train accuracy:
train_y_pred = tree_clf.predict(train_x)

print(accuracy_score(train_y, train_y_pred))

0.7453016815034619


In [29]:
#Test accuracy:
test_y_pred = tree_clf.predict(test_x)

print(accuracy_score(test_y, test_y_pred))

0.565


In [30]:
from sklearn.metrics import confusion_matrix

#Test confusion matrix
confusion_matrix(test_y, test_y_pred)

array([[831, 534],
       [597, 638]])

In [31]:
from sklearn.metrics import classification_report

print(classification_report(test_y, test_y_pred))

              precision    recall  f1-score   support

           0       0.58      0.61      0.60      1365
           1       0.54      0.52      0.53      1235

    accuracy                           0.56      2600
   macro avg       0.56      0.56      0.56      2600
weighted avg       0.56      0.56      0.56      2600



# Decision Tree Model 2:

In [32]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_grid = {'max_depth': randint(low=1, high=100), 
              'min_samples_leaf': randint(low=1, high=100)}

tree_gs = RandomizedSearchCV(DecisionTreeClassifier(), param_grid, 
                             n_iter=20, cv=30, verbose=1,
                             scoring='accuracy',
                             return_train_score=True)

tree_gs.fit(train_x, train_y)

Fitting 30 folds for each of 20 candidates, totalling 600 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 600 out of 600 | elapsed:  1.2min finished


RandomizedSearchCV(cv=30, estimator=DecisionTreeClassifier(), n_iter=20,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7faba799ea00>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7faba79865e0>},
                   return_train_score=True, scoring='accuracy', verbose=1)

In [33]:
cvres = tree_gs.cv_results_

for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(mean_score, params)

0.5915004308312605 {'max_depth': 3, 'min_samples_leaf': 42}
0.5507925669414232 {'max_depth': 58, 'min_samples_leaf': 2}
0.5677591897120746 {'max_depth': 98, 'min_samples_leaf': 63}
0.5831162919247589 {'max_depth': 46, 'min_samples_leaf': 44}
0.6045147864540149 {'max_depth': 4, 'min_samples_leaf': 48}
0.5491163894714595 {'max_depth': 88, 'min_samples_leaf': 5}
0.5712123429091679 {'max_depth': 40, 'min_samples_leaf': 69}
0.5939602334617697 {'max_depth': 6, 'min_samples_leaf': 72}
0.5634630704449755 {'max_depth': 60, 'min_samples_leaf': 8}
0.5727080589832383 {'max_depth': 34, 'min_samples_leaf': 58}
0.5722130094782878 {'max_depth': 90, 'min_samples_leaf': 58}
0.5710749646393211 {'max_depth': 72, 'min_samples_leaf': 29}
0.5464826285584224 {'max_depth': 48, 'min_samples_leaf': 5}
0.5733795054382286 {'max_depth': 62, 'min_samples_leaf': 52}
0.5936350777934934 {'max_depth': 2, 'min_samples_leaf': 91}
0.5715570079175406 {'max_depth': 75, 'min_samples_leaf': 25}
0.5857199759384806 {'max_depth':

In [34]:
#Best parameter set
tree_gs.best_params_

{'max_depth': 4, 'min_samples_leaf': 48}

In [35]:
tree_gs.best_estimator_

DecisionTreeClassifier(max_depth=4, min_samples_leaf=48)

In [36]:
#Train accuracy:
train_y_pred = tree_gs.best_estimator_.predict(train_x)

print(accuracy_score(train_y, train_y_pred))

0.6160567095285197


In [37]:
#Test accuracy:
test_y_pred = tree_gs.best_estimator_.predict(test_x)

print(accuracy_score(test_y, test_y_pred))

0.6073076923076923


In [38]:
#Confusion matrix
confusion_matrix(test_y, test_y_pred)

array([[1014,  351],
       [ 670,  565]])